### Setup Environment:

In [4]:
from src.get_dataset import get_dataset
from src.data_loader import CustomLabeledImageDataset

In [5]:
# Constants:
DATASET = 'data/'
DOWNLOAD = False

#### Read csv file:

In [7]:
df = get_dataset("data/", download=DOWNLOAD, info=False)

loading csv file in data//labels.csv


### Load the Dataset